In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report
import pickle
import seaborn as sns; 
sns.set(style ='white', color_codes = True)

data = pd.read_csv('C:\\Users\\aibar\\Desktop\\mlat1\\train.csv')
data.columns = map(str.lower, data.columns)

# Handle missing values (for simplicity, filling NaNs with 0)
data.fillna(0, inplace=True)

# store columns with specific data type
integer_columns = data.select_dtypes(include=['int64']).columns
float_columns = data.select_dtypes(include=['float64']).columns
object_columns = data.select_dtypes(include=['object']).columns


integer_columns

float_columns


data = data[['gp', 'ftm', 'fta', 'twopm', 'twopa', 'tpm', 'tpa', 'year', 'min_per', 'ortg', 'usg', 'efg', 'ts_per', 'orb_per', 'drb_per',
       'ast_per', 'to_per', 'ft_per', 'twop_per', 'tp_per', 'blk_per',
       'stl_per', 'ftr', 'porpag', 'adjoe', 'pfr', 'rec_rank', 'ast_tov',
       'rimmade', 'rimmade_rimmiss', 'midmade', 'midmade_midmiss', 'rim_ratio',
       'mid_ratio', 'dunksmade', 'dunksmiss_dunksmade', 'dunks_ratio', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'drafted']]


# Splitting features and target variable
X = data.drop(['drafted'], axis=1)
y = data['drafted']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


from imblearn.over_sampling import RandomOverSampler

# Create the RandomOverSampler object
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Resample the training data
X_train_balanced, y_train_balanced = oversampler.fit_resample(X_train_scaled, y_train)


from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, f_classif

# Initialize the SelectKBest object with the f_classif scoring function (appropriate for classification tasks)
k_best = SelectKBest(score_func=f_classif, k=5)

# Fit the k-best selector to your training data and transform it to select the top 5 features
X_train_top5 = k_best.fit_transform(X_train_balanced, y_train_balanced)

# Get the indices of the selected features
selected_feature_indices = k_best.get_support(indices=True)

# Get the names of the selected features
selected_features = X_train.columns[selected_feature_indices]

# Print the names of the selected features
print("Selected Features:")
print(selected_features)


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Use the selected features to subset your data
X_train_selected = X_train_top5
X_test_selected = X_test_scaled[:, selected_feature_indices]


from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler

# define the models
lr = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
xgb = xgb.XGBClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

def plot_roc_curve(model, X_test, y_test):
    y_pred_proba = model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()


models = [lr, rf, xgb, gb]

    
best_model = None
best_auroc = 0.0  # Initialize with a very low value

for model in models:
    scores = cross_val_score(model, X_train_balanced, y_train_balanced, cv=5, scoring='roc_auc')
    mean_auroc = np.mean(scores)
    print(type(model).__name__)
    print("Mean ROC AUC:", mean_auroc)
    
    # Fit the model and calculate predicted probabilities
    model.fit(X_train_selected, y_train_balanced)
    y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
    
    # Calculate ROC AUC score
    model_auroc = roc_auc_score(y_test, y_pred_proba)
    print("Test ROC AUC score:", model_auroc)
    
    if model_auroc > best_auroc:
        best_auroc = model_auroc
        best_model = model
        
    print("==============================")

# Print the best model with the highest AUROC score
print("Best model:", type(best_model).__name__, "with AUROC:", best_auroc)

from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Get the predicted probabilities for class 1 (drafted)
y_pred_prob = rf.predict_proba(X_test_selected)[:, 1]

# Calculate ROC curve and AUC score
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()


C:\Users\aibar\AppData\Local\Temp\ipykernel_6360\1735089047.py:15: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:\\Users\\aibar\\Desktop\\mlat1\\train.csv')


Selected Features:
Index(['porpag', 'pick', 'dporpag', 'stops', 'pts'], dtype='object')


C:\Users\aibar\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\aibar\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LogisticRegression
Mean ROC AUC: 0.9958593150521251
Test ROC AUC score: 0.9937440956325848


In [ ]:
# Load and preprocess your test dataset
test = pd.read_csv('C:\\Users\\aibar\\Desktop\\mlat1\\test.csv') 
test.columns = map(str.lower, test.columns)
test.fillna(0, inplace=True)

In [ ]:
test_data = test[['gp', 'ftm', 'fta', 'twopm', 'twopa', 'tpm', 'tpa', 'year', 'min_per', 'ortg', 'usg', 'efg', 'ts_per', 'orb_per', 'drb_per',
       'ast_per', 'to_per', 'ft_per', 'twop_per', 'tp_per', 'blk_per',
       'stl_per', 'ftr', 'porpag', 'adjoe', 'pfr', 'rec_rank', 'ast_tov',
       'rimmade', 'rimmade_rimmiss', 'midmade', 'midmade_midmiss', 'rim_ratio',
       'mid_ratio', 'dunksmade', 'dunksmiss_dunksmade', 'dunks_ratio', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts','player_id']]


In [ ]:
# Splitting features for test data
X_test_data = test_data.drop(['player_id'], axis=1)  # Remove any identifier column that's not a feature

# Standardize the test features using the same scaler
X_test_data_scaled = scaler.transform(X_test_data)

X_test_data_scaled = X_test_data_scaled[:, selected_feature_indices]

In [ ]:
# Make predictions on the test data
y_pred_prob_test = rf.predict_proba(X_test_data_scaled)[:, 1]  # Predicted probabilities for class 1 (drafted)

# Visualize predicted probabilities
plt.figure(figsize=(8, 6))
plt.hist(y_pred_prob_test, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Predicted Probability of Being Drafted')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.show()